In [54]:
import pandas as pd
import xgboost as xgb
import numpy as np
from scipy.stats import uniform as sp_rand
from scipy.stats import randint as sp_randint
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## 6. Model

In [5]:
# data
dt_model = pd.read_csv("../../data/Mercedes_Benz_Greener_Manufacturing/data/dt_all_preprocess.csv")

In [45]:
# r_2 for xgboost
def r_2(preds, dtrain):
    labels = dtrain.get_label()
    return 'score', r2_score(labels, preds)

# make scorer_r2
scorer_r2 = make_scorer(r2_score)

In [13]:
# X, y, ID
X_train_all = dt_model.loc[dt_model["IsTrainTest"] == "train"].drop(["ID", "y", "IsTrainTest"], axis = 1)
X_test = dt_model.loc[dt_model["IsTrainTest"] == "test"].drop(["ID", "y", "IsTrainTest"], axis = 1)
y_train_all = dt_model.loc[dt_model["IsTrainTest"] == "train"].y.values
y_test = dt_model.loc[dt_model["IsTrainTest"] == "test"].y.values
ID_train_all = dt_model.loc[dt_model["IsTrainTest"] == "train"].ID.values
ID_test = dt_model.loc[dt_model["IsTrainTest"] == "test"].ID.values

### 6.1 xgboost

In [134]:
# params_xgb
params_xgb = {
    "max_depth" : sp_randint(2, 6)
    , "learning_rate" : sp_rand(.0001, .0999) # .0001 to .01
    , "n_estimators" : sp_randint(200, 2000)
    , "booster" : ["gbtree", "gblinear", "dart"]
    , "gamma" : sp_rand(0.1, 4.9) # .1 to 5
    , "min_child_weight" : sp_randint(2, 20)
    , "subsample" : sp_rand(.5, .4) # .5 to .9
    , "colsample_bytree" : sp_rand(.5, .4) # .5 to .9
    , "colsample_bylevel" : sp_rand(.5, .4) # .5 to .9
    , "reg_alpha" : sp_rand(.1, .4) # .1 to .5
    , "reg_lambda" : sp_rand(.1, .4) # .1 to .5
}

In [135]:
# xgb
model_xgb = xgb.XGBRegressor(n_jobs = 6, base_score = np.mean(y_train_all))

In [136]:
# random search
randSearch_xgb = RandomizedSearchCV(model_xgb, params_xgb, scoring = scorer_r2
                                    , n_iter = 10, n_jobs = 6, cv = 3, verbose = 1)

In [ ]:
randSearch_xgb.fit(X_train_all, y_train_all)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
